<a href="https://colab.research.google.com/github/VitorLima2521/Twitter-Emotion-Classification/blob/main/Twitter_Emotion_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
parulpandey_emotion_dataset_path = kagglehub.dataset_download('parulpandey/emotion-dataset')

print('Data source import complete.')


100%|██████████| 715k/715k [00:00<00:00, 1.10MB/s]

Extracting files...
Data source import complete.


![](https://i.imgur.com/KOjXRJg.jpg)

### <b><span style='color:#F1A424'>Test Classification</span></b>

- A classificação de texto é uma das tarefas mais comuns em PNL
- Ele pode ser usado para uma ampla gama de aplicações (por exemplo, marcar o feedback do cliente em categorias, encaminhar tickets de suporte de acordo com o idioma)
- Outro tipo comum de problema de classificação de texto é **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">sentiment analysis</mark>** que visa **identificar a polaridade** de um determinado texto (+/-)

### <b><span style='color:#F1A424'>Nossa Tarefa</span></b>

- Precisamos construir um sistema que seja capaz de identificar automaticamente estados emocionais (por exemplo, raiva, alegria) que as pessoas expressam sobre o produto da sua empresa no Twitter.
- Para esta tarefa, usaremos uma variante de **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">BERT</mark>**; **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>**, a principal vantagem deste modelo é que ele é muito menor que **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">BERT</mark>** (ou seja, mais eficiente), mas é capaz de atingir um desempenho comparável
- Usaremos três bibliotecas principais do **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Hugging Face</mark>** ecossistema: **<span style='color:#FFC300'>Datasets</span>**, **<span style='color:#FFC300'>Tokenizers</span>** & **<span style='color:#FFC300'>Transformers</span>**

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>1 |</span></b> <b>O DATASET</b></div>
    
- Muitos conjuntos de dados que envolvem análise de sentimentos são problemas de classificação binária.
- Neste conjunto de dados, temos 6 sentimentos diferentes, o que significa que trataremos este problema como um problema de classificação multiclasse.

In [ ]:
!ls /kaggle/input/emotion-dataset/

ls: cannot access '/kaggle/input/emotion-dataset/': No such file or directory


In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import warnings; warnings.filterwarnings('ignore')

def show_panel(df):
    return pn.widgets.Tabulator(df.head(20),
                    show_index=False,
                    pagination='local',
                         page_size=10)


pn.extension('tabulator')
pn.widgets.Tabulator.theme = 'bootstrap'

validation = pd.read_csv('/kaggle/input/emotion-dataset/validation.csv')
train = pd.read_csv('/kaggle/input/emotion-dataset/training.csv')
test = pd.read_csv('/kaggle/input/emotion-dataset/test.csv')

print('Dataset information:')
print(f'Training data: {train.shape}')
print(f'Validation data: {validation.shape}')
print(f'Test data: {test.shape}')

In [ ]:
show_panel(train)

### <b><span style='color:#F1A424'>Combinar DataFrames em Dataset</span></b>
- Pode ser mais intuitivo utilizar o conjunto de dados do **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">HuggingFace</mark>**'s Dataset

In [ ]:
from datasets import Dataset,DatasetDict,Features,Value,ClassLabel

# Don't forget the class label data
class_names = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
ft = Features({'text': Value('string'), 'label': ClassLabel(names=class_names)})

# Combine Multiple Datasets
emotions = DatasetDict({
    "train": Dataset.from_pandas(train,features=ft),
    "test": Dataset.from_pandas(test,features=ft),
    "validation": Dataset.from_pandas(validation,features=ft)
    })

# Convert a single DataFrame to a Dataset
# emotions = Dataset.from_pandas(train,features=ft)

emotions

### <b><span style='color:#F1A424'>Selecionando um subconjunto</span></b>
- Trabalharemos com o conjunto de dados de treinamento e validação neste problema.
- Vamos mostrar algumas características úteis da classe **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Dataset</mark>**

In [ ]:
# Training Data
train_ds = emotions["train"]
train_ds

In [ ]:
# Get First 5 Entries in Dictionary Format (Group them)
train_ds[:5]

### <b><span style='color:#F1A424'>Conversão de conjunto de dados em DataFrame</span></b>
- Sempre que precisamos de um pandas `DataFrame`, por exemplo, para visualizações, Sempre que precisamos de um DataFrame pandas, por exemplo, para visualizações, podemos utilizar o método ''  `Dataset`'' `.set_format`

In [ ]:
# Convert Dataset to DataFrame (don't forget to reset)
emotions.set_format(type="pandas")
df = emotions["train"][:]
show_panel(df)

### <b><span style='color:#F1A424'>Adicionando dados de rótulo</span></b>
- Certificamo-nos de não esquecer os `label_names` ao converter de `DataFrame` para `Dataset`
- Podemos converter valores **numéricos** em valores de **string** usando o método `int2str`

In [ ]:
# Add label data to dataframe
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
show_panel(df)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>2 |</span></b> <b>DISTRIBUIÇÃO DE CLASSES</b></div>

- Nosso conjunto de dados tem 6 classes `joy`, `sadness`, `anger`, `fear`, `love` e `surprise`; problema multiclasse

In [ ]:
import plotly.express as px

px.bar(df['label_name'].value_counts(ascending=True),template='plotly_white')

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>3 |</span></b> <b>DURAÇÃO DE CADA TWEET</b></div>

- Para aplicações que utilizam **DistilBERT**, o tamanho máximo do contexto é **512 tokens**
- A maioria dos tweets tem entre **10 e 20 palavras**, o que se enquadra perfeitamente nesse limite

In [ ]:
df["Words Per Tweet"] = df["text"].str.split().apply(len)

px.box(df,y='Words Per Tweet',
       color='label_name',
       template='plotly_white')

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>4 |</span></b> <b>TOKENIZAÇÃO</b></div>

- Assim como outros modelos, o **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>** não pode receber strings python brutas como entrada
- Em vez disso, precisamos dividir as strings em subgrupos chamados **tokens** e codificá-los como **vetores numéricos**
- Vamos considerar dois tipos de abordagens de **tokenização**: **<span style='color:#FFC300'>caracteres</span>** & **<span style='color:#FFC300'>palavras</span>** tokenização

### <b><span style='color:#F1A424'> 4.1 | </span>Character Tokenização </b>

A abordagem de tokenização mais simples é a **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">character tokenisation</mark>**, podemos usar a classe de lista interna do Python

In [ ]:
text = 'Tokenisation of text is a core task of NLP.'
tokenised_text = list(text)

# Character Tokenised list
print(f'Number of tokens: {len(tokenised_text)}')
print(tokenised_text)

- Converter cada caractere em um inteiro (numericalização)
- `token2idx` nos dá um mapeamento de cada caractere no **vocabulário** para um inteiro único

In [ ]:
# Mapping Vecabulary dictionary
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenised_text)))}

print(f'Length of vocabulary: {len(token2idx)}')
print(token2idx)

### <b><span style='color:#F1A424'>Reconstruindo texto</span></b>
- Uma vez que temos um dicionário de vocabulário, podemos reconstruir

In [ ]:
# Let's represent text in numerical format
input_ids = [token2idx[token] for token in tokenised_text]

print(f'{len(input_ids)} characters')
print(input_ids)

### <b><span style='color:#F1A424'>Convert to OHE</span></b>

- O último passo é converter `input_ids` em um tensor 2D de vetores one-hot, vamos usar o pytorch abaixo
- Os vetores One-Hot são frequentemente usados ​​em aplicações de ML para codificar **dados categóricos** (ordinais ou nominais)
- Para cada um dos 42 tokens de entrada, agora temos um vetor one-hot com 18 dimensões (tamanho do vocabulário)

In [ ]:
import torch
import torch.nn.functional as F

inputs_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(inputs_ids,num_classes = len(token2idx))
print(f'OHE size: {one_hot_encodings.shape}')

### <b><span style='color:#F1A424'>Desvantagens da Tokenização de Caracteres</span></b>

- A tokenização em nível de caractere ignora qualquer estrutura no texto e trata toda a sequência como um fluxo de caracteres
- Isso ajuda a lidar com erros ortográficos e palavras repetidas, mas a principal desvantagem é que as estruturas linguísticas precisam ser aprendidas a partir dos dados
- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Character tokenisation</mark>** raramente é usado na prática, em vez disso, alguma estrutura do texto é preservada se utilizarmos **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Word Tokenisation</mark>**

### <b><span style='color:#F1A424'> 4.2 | </span>Tokenização de palavras </b>

- Em vez de dividir o texto em caracteres, podemos dividi-lo em palavras e mapear cada palavra para um inteiro.
- A forma mais simples de tokenização é se utilizarmos o método de divisão de classe de string embutido do Python
- Ao contrário **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Character tokenisation</mark>**, se tivermos declinações, conjugações e erros ortográficos, o tamanho do dicionário de vocabulário pode aumentar muito rapidamente.
- **Vocabulários** maiores são um problema, porque exigem que o modelo tenha um excesso de parâmetros (o que é ineficiente)


- É comum selecionar as 100.000 palavras mais comuns no corpus
- palavras que não fazem parte do vocabulário são classificadas como desconhecidas e mapeadas para um token**<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">UNK</mark>** compartilhado
- No entanto, pode potencialmente perder algumas informações importantes durante o processo de tokenização, uma vez que o modelo não possui informações sobre palavras associadas a **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">UNK</mark>**

In [ ]:
tokenised_text = text.split()
print(tokenised_text)

['Tokenisation', 'of', 'text', 'is', 'a', 'core', 'task', 'of', 'NLP.']


### <b><span style='color:#F1A424'> 4.3 | </span>Tokenização de subpalavra </b>

- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Subword tokenization</mark>** é combinar os melhores aspectos da tokenização de  **<span style='color:#FFC300'>caracteres</span>** & **<span style='color:#FFC300'>palavra</span>**
- A principal característica distintiva de **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Subword tokenization</mark>** é que ele é aprendido a partir de um corpus de pré-treinamento usando uma mistura de regras estatísticas e algoritmos


- Existem vários **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Subword tokenization</mark>**algoritmos comumente usados ​​em NLP
    - vamos começar com `WordPiece`, que é usado pelo `BERT` e `DistilBERT` tokenizadores
    

- `AutoTokenizer` a classe nos permite carregar rapidamente o tokenizador associado a um modelo pré-treinado
- Ou podemos carregar o Tokeniser manualmente a partir de `transformers.DistilBertTokenizer`



In [ ]:
from transformers import AutoTokenizer

text = 'Tokenisation of text is a core task of NLP.'

# Load parameters of the tokeniser
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Show tokeniser information
tokenizer

# Or we can load the Tokeniser manually `transformers.DistilBertTokenizer`

# from transformers import DistilBertTokenizer

# model_ckpt = "distilbert-base-uncased"
# distilbert_tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
# distilbert_tokenizer

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
print('')
print(f'Vocab size: {tokenizer.vocab_size}')
print(f'Max length: {tokenizer.model_max_length}')
print(f'Tokeniser model input names: {tokenizer.model_input_names}')


Vocab size: 30522
Max length: 512
Tokeniser model input names: ['input_ids', 'attention_mask']


In [ ]:
print('Encoded text')
encoded_text = tokenizer(text)
print(encoded_text,'\n')

print('Tokens')
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens,'\n')

print('Convert tokens to string')
print(tokenizer.convert_tokens_to_string(tokens),'\n')

Encoded text
{'input_ids': [101, 19204, 6648, 1997, 3793, 2003, 1037, 4563, 4708, 1997, 17953, 2361, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

Tokens
['[CLS]', 'token', '##isation', 'of', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', '##p', '.', '[SEP]'] 

Convert tokens to string
[CLS] tokenisation of text is a core task of nlp. [SEP] 



In [ ]:
emotions.reset_format()

### <b><span style='color:#F1A424'> 4.4 | </span>Tokenização de todo o conjunto de dados </b>

- Ao lidar com texto de tamanhos diferentes, o tokenizador irá **<span style='color:#FFC300'>pad</span>** frases de comprimento insuficiente se **padding** é selecionado
- O **comprimento máximo** dos dados tokenizados será o **comprimento do tweet mais longo** (por exemplo, 2ª linha)
- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Attention mask</mark>** ajuda o modelo a entender quais partes da frase ignorar

In [ ]:
# Tokenisation function
def tokenise(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

# Show the tokenised ids
ex_tokenised = tokenise(emotions["train"][:2])

In [ ]:
# Show attention mask
ex_tokenised['attention_mask']

In [ ]:
# apply to the entire dataset (train,test and validation dataset)
emotions_encoded = emotions.map(tokenise, batched=True, batch_size=None)
print(emotions_encoded["train"].column_names)

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>5 |</span></b> <b>TRENANDO UM CLASSIFICADOR DE TEXTO</b></div>

- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>** esse modelos **são pré-treinados para prever palavras mascaradas em uma frase de texto.**
- Não podemos usar esses modelos de linguagem diretamente para classificação de texto, sendo necessárias algumas pequenas modificações.


- Primeiramente, o texto é tokenizado, representado usando vetores one-hot chamados **token encodings**
- O tamanho do vocabulário do tokenizador determina a dimensão da próxima codificação (geralmente 20-200k)
- Em seguida, essas codificações de token são convertidas em embeddings de token (vetores que residem em um espaço dimensional inferior).
- Os embeddings de token são então passados ​​pelas camadas de blocos do codificador para produzir um estado oculto para cada token de entrada.
*Para o objetivo pré-treinado de modelagem de linguagem, cada estado oculto é alimentado a uma camada que prevê os tokens de entrada mascarados.
* Para a tarefa de classificação, substituímos a camada de modelagem de linguagem por uma camada de classificação.
Temos duas opções para treinar esse modelo em nosso conjunto de dados:

Temos duas opções para treinar esse modelo em nosso conjunto de dados:
- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Feature Extraction</mark>** : Usamos os estados ocultos como características e apenas treinamos o classificador com base neles, sem modificar o modelo pré-treinado.
- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Fine Tuning</mark>** : Treinamos todo o modelo, de ponta a ponta, que posteriormente também atualiza os parâmetros do modelo pré-treinado


### <b><span style='color:#F1A424'> 5.1 | </span>Transformers as feature extractors </b>

### **<span style='color:#F1A424'>Modelos pré-treinados</span>**

- Usaremos outra classe automatica `AutoModel`, similar ao `AutoTokenizer`
- `AutoModel` possui o métado `from_pretrained` para carregar os pesos de um modelo prétreinado
- A classe `AutoModel` converte as codificações de tokens em embeddings e os alimenta atrav´ws da pilha do codificador para retornar **estados ocultos**

In [ ]:
import warnings; warnings.filterwarnings('ignore')
from transformers import AutoModel
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

### **<span style='color:#F1A424'>Extraindo o último estado ocultos</span>**
estado oculto de uma única string **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">estado oculto</mark>**
- Primeiro, codifica a string e converte os tokens em tesores PyToch
- o tensor resultante tem o formato **[batch_size,n_tokens]**
- Tnedo as codificações como tensores, a etapas é colocá-las no mesmo dispositivo que o modelo e passar as entradas:

In [ ]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
print(f"Input tensor shape: {inputs['input_ids'].size()}")

- Dependendo da configuração do modelo, o modelo pode conter vários objetos (**<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Hidden states</mark>**, **losses**, **attentions**, ...)
- O modelo atual (`distilbert-base-uncased`) retorna apenas um atributo, que é o `last_hidden_state`

In [ ]:
inputs = {k:v.to(device) for k,v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

- O tensor de estado oculto (`last_hidden_state`) tem o tamanho: **[batch_size,n_tokens,hidden_dim]**
- ou seja, um vetor de 768 dimensões é retornado para cada um dos 6 tokens de entrada


- Para **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">classificação de tarefas</mark>**
    - é prática comum usar apenas o **estado oculto associado ao token [CLS]** como recurso de entrada

In [ ]:
print(outputs.last_hidden_state.size())
print(outputs.last_hidden_state[:,0].size())

### **<span style='color:#F1A424'>Extraindo o último estado oculto do conjunto de dados</span>**

- Sabemos como obter o último estado oculto para uma única string, vamos repetir o processo para todo o conjunto de dados usando `extract_hidden_states`

In [ ]:
def extract_hidden_states(batch):

    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state

    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
emotions_encoded

In [ ]:
# Extract last hidden states (faster w/ GPU)
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)
emotions_hidden["train"].column_names

### **<span style='color:#F1A424'>Criando a Matriz de Recursos</span>**

- Nós temos **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">hidden states</mark>** associado a cada tweet, agora vamos treinar o classificador
- Para fazer isso, precisamos da matriz de recursos para que possamos utilizá-la como entrada no modelo de aprendizado de máquina

In [ ]:
X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
print(f'Training Dataset: {X_train.shape}')
print(f'Validation Dataset {X_valid.shape}')

In [ ]:
# Let's check our dataset
X_train

### **<span style='color:#F1A424'>Vizualizando os dados de treino</span>**

- Podemos visualizar cada distribuição de classe que o modelo precisará separar em **<span style='color:#FFC300'>espaço de dimensão inferior</span>** (projeções em um espaço de menor dimensão)
- Temos muitas **categorias sobrepostas** no espaço dimensional inferior (não significa que o modelo não será capaz de classificá-las em **<span style='color:#FFC300'>espaço dimensional superior</span>**)
- Se forem separáveis ​​no espaço projetado, provavelmente serão separáveis ​​em **<span style='color:#FFC300'>espaço dimensional superior</span>**
- Utilizaremos um **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">manifold learning</mark>** modelo não supervisionado **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">TSNE</mark>** (Vai demorar um pouco)

In [ ]:
import warnings; warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

# Scale the data
X_scaled = MinMaxScaler().fit_transform(X_train)

# lower dimension transformation
model = TSNE(n_components=2).fit(X_scaled)

# Create a df of 2D embeddings
df_embedding = pd.DataFrame(model.embedding_, columns=["X", "Y"])
df_embedding["label"] = y_train

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')

fig, axes = plt.subplots(2, 3, figsize=(15,10))
axes = axes.flatten()
labels = emotions["train"].features["label"].names

for i, label in enumerate(labels):

    dict_embedding_sub = dict(tuple(df_embedding.groupby('label')))
    df_embedding_sub = dict_embedding_sub[i]

    axes[i].scatter(df_embedding_sub["X"],
                    df_embedding_sub["Y"],
                    lw=1,ec='k',alpha=0.2)

    axes[i].set_title(f'{label}')

plt.tight_layout()
plt.show()

Neste gráfico podemos ver alguns padrões claros:
- Para **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">positive emotions</mark>** (**<span style='color:#FFC300'>joy</span>** and **<span style='color:#FFC300'>love</span>**) estão bem separados do **<span style='color:#FFC300'>negative emotions</span>** e também compartilham um espaço semelhante
- **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">Negative emotions</mark>** (**<span style='color:#FFC300'>sadness</span>**, **<span style='color:#FFC300'>anger</span>**, and **<span style='color:#FFC300'>fear</span>**) todos ocupam regiões muito semelhantes com distribuições ligeiramente variáveis **<span style='color:#FFC300'>espaço de dimensão inferior</span>**
- Finalmente, **<span style='color:#FFC300'>surprise</span>** está um pouco espalhado por todo o **<span style='color:#FFC300'>espaço de dimensão inferior</span>**

### **<span style='color:#F1A424'>Treinamento de um modelo de linha de base</span>**

- Vamos usar estes **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">hidden states</mark>** para treinar um modelo **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">logistic regression</mark>**
- Estamos lidando com um conjunto de dados multiclasse desbalanceado, então nosso modelo pode parecer melhor do que aleatório, mas na verdade é melhor, vamos comparar com um `DummyClassifier`
- `DummyClassifier` pode ser usado para construir um classificador com heurísticas simples (escolhendo a classe majoritária/sempre desenhando uma classe aleatória),
- Vamos escolher os mais frequentes (`strategy="most_frequent"`) então temos um modelo de referência para comparação

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
print(f'accuracy: {dummy_clf.score(X_valid, y_valid)}')

In [ ]:
from sklearn.linear_model import LogisticRegression as LR

# We increase `max_iter` to guarantee convergence
lr_clf = LR(max_iter = 2000)
lr_clf.fit(X_train, y_train)
y_preds = lr_clf.predict(X_valid)
print(f'accuracy: {lr_clf.score(X_valid, y_valid)}')

### <b><span style='color:#F1A424'>Matriz de Confusão</span></b>

- Nosso  **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">logistic regression</mark>** modelo com `DistilBERT` **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">embeddings</mark>**é significativamente melhor do que a linha de base `DummyClassifier`
- Vamos verificar o **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">confusion matrix</mark>** do modelo **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">logistic regression</mark>**

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_model, y_true, labels):
    cm = confusion_matrix(y_true,y_model,normalize='true')
    fig, ax = plt.subplots(figsize=(7,7))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(ax=ax, colorbar=False)
    plt.title("Confusion matrix")
#     plt.axis('off')
    plt.grid(False)
    plt.show()

plot_confusion_matrix(y_preds, y_valid, labels)

Do **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">confusion matrix</mark>** podemos dizer que:

- **<span style='color:#FFC300'>anger</span>**, **<span style='color:#FFC300'>fear</span>** & **<span style='color:#FFC300'>surprise</span>** frequentemente confundido com **<span style='color:#FFC300'>sadness</span>** podemos dizer que (0,29, 0,17 e 0,14) (observação que fizemos ao visualizar os embeddings)
- **<span style='color:#FFC300'>love</span>** & **<span style='color:#FFC300'>surprise</span>** são frequentemente confundidos com **<span style='color:#FFC300'>joy</span>** (0.37 & 0.46)

### <b><span style='color:#F1A424'> 5.2 | </span>Transformadores de ajuste fino </b>

- Com a abordagem de ajuste fino, não usamos o **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">hidden states</mark>** como recursos fixos, em vez disso, nós os treinamos a partir de um determinado estado do modelo
- Isso requer que a cabeça de classificação seja diferenciável (rede neural para classificação)

### <b><span style='color:#F1A424'>Carregando um modelo pré-treinado</span></b>

- Nós carregaremos o mesmo **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>** modelo usando `model_ckpt` **"distilbert-base-uncased"**
- Desta vez, porém, estaremos carregando `AutoModelForSequenceClassification` (nós usamos `AutoModel` quando extraímos recursos de incorporação)
- `AutoModelForSequenceClassification` modelo tem um **<span style='color:#FFC300'>classification head</span>** sobre as saídas do modelo pré-treinado
- Precisamos apenas especificar o **<span style='color:#FFC300'>number of labels</span>** o modelo tem que prever `num_labels`

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 6

model_ckpt = "distilbert-base-uncased"
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt,
                          num_labels=num_labels)
         .to(device))

### <b><span style='color:#F1A424'>Definindo as métricas de desempenho</span></b>
- Nós monitoraremos o `F1 score`  & `accuracy`, a função deve ser passada na classe `Trainer`



In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

### <b><span style='color:#F1A424'>Parâmetros de treinamento</span></b>
- Em seguida, precisamos definir os **parâmetros de treinamento** do modelo, o que pode ser feito usando `TrainingArguments`
-Vamos treinar o **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>** modelo para **3 iterações** com uma **taxa de aprendizagem de 2e-5** e um **tamanho de lote de 64**

In [ ]:
from transformers import Trainer, TrainingArguments

bs = 64 # batch size
logging_steps = len(emotions_encoded["train"]) // bs
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=3,             # number of training epochs
                                  learning_rate=2e-5,             # model learning rate
                                  per_device_train_batch_size=bs, # batch size
                                  per_device_eval_batch_size=bs,  # batch size
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  report_to="none",
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

### <b><span style='color:#F1A424'>Modelo de treino</span></b>
-Com os argumentos de treinamento definidos, precisamos definir o `Trainer` e comece a treinar com o método `train()`

In [ ]:
import os
from transformers import Trainer
os.environ['WANDB_DISABLED'] = 'true'

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train()

In [ ]:
# Predict on Validation Dataset
pred_output = trainer.predict(emotions_encoded["validation"])
pred_output

In [ ]:
print(f'Output Predition: {pred_output.predictions.shape}')
print(pred_output.predictions)

In [ ]:
# Decode the predictions greedily using argmax (highest value of all classes)
y_preds = np.argmax(pred_output.predictions,axis=1)
print(f'Output Prediction:{y_preds.shape}')
print(f'Predictions: {y_preds}')

In [ ]:
# Show metrics of last iteration
pred_output.metrics

In [ ]:
plot_confusion_matrix(y_preds,y_valid,labels)

Do **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">confusion matrix</mark>** podemos dizer que:
- O **fine-tune** abordagem (usando **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>**)  tem um desempenho muito melhor do que a simples extração **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">embedding</mark>** dados e treiná-los em um modelo de ML separado
- **<span style='color:#FFC300'>love</span>** ainda é frequentemente confundido com **<span style='color:#FFC300'>joy</span>** (0.08), mas muito menos que a primeira abordagem
- **<span style='color:#FFC300'>surprise</span>** é frequentemente confundido com **<span style='color:#FFC300'>joy</span>** bem como (0,09) ou medo (0,10), ambos também muito menores que a primeira abordagem

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>6 |</span></b> <b>ANÁLISE DE ERROS DO MODELO</b></div>

### <b><span style='color:#F1A424'>Mapeamento de Valor de Perda</span></b>

Deveríamos investigar um pouco mais a previsão dos nossos modelo
- Uma técnica simples, mas poderosa, é classificar a validação por **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">model loss</mark>**
- Podemos escrever uma função que retorna o **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">model loss</mark>**, juntamente com o rótulo previsto `forward_pass_with_label`

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):

    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),
                             reduction="none")

    # Place outputs on CPU for compatibility with other dataset columns
    return {"loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()}

# Convert our dataset back to PyTorch tensors
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
# Compute loss values
emotions_encoded["validation"] = emotions_encoded["validation"].map(forward_pass_with_label,
                                                                    batched=True,
                                                                    batch_size=16)

### <b><span style='color:#F1A424'>Converter para DataFrame</span></b>

- Crie um DataFrame com o texto, perdas, rótulos previstos/verdadeiros

In [ ]:
emotions_encoded.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["validation"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"].apply(label_int2str))

- Agora podemos classificar `emotions` codificado pelas perdas em ordem ascendente/descendente
- Vamos analisar as amostras de dados com as **maiores perdas** (podemos ver que valores de perdas elevadas estão associados a previsões erradas)

In [ ]:
show_panel(df_test.sort_values("loss", ascending=False))

In [ ]:
show_panel(df_test.sort_values("loss", ascending=True))

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;overflow:hidden;background-color:#3b3745"><b><span style='color:#F1A424'>7 |</span></b> <b>USANDO NOSSO MODELO</b></div>

-Nós treinamos o modelo utilizando `AutoModelForSequenceClassification` que adicionou uma cabeça de classificação à base modelo **<mark style="background-color:#FFC300;color:white;border-radius:5px;opacity:0.7">DistilBERT</mark>**
- Podemos utilizar o `pipeline` método quando precisamos fazer previsões de modelo em novos dados não semente
- Digamos que temos novos dados não vistos:
    - 'Assisti a um filme ontem à noite, foi brilhante'

In [ ]:
# Save the model
trainer.save_model()

In [ ]:
from transformers import pipeline

# load from previously saved model
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-emotion")

# New unseen by model data
new_data = 'I watched a movie last night, it was quite brilliant'

- Nosso modelo prevê `new_data` ser classificado como **rótulo 1** (**alegria**)

In [ ]:
preds = classifier(new_data, return_all_scores=True)
preds

In [ ]:
preds[0]

In [ ]:
df_preds = pd.DataFrame(preds)
px.bar(x=labels,y=100*df_preds['score'],template='plotly_white')